In [4]:
:set -XNoImplicitPrelude
:set -XOverloadedStrings
:set -XLambdaCase

In [5]:
import Papa
import Control.Monad.State
import qualified Prelude as P (read, undefined)
import qualified Data.IntMap as IM
import Data.Monoid


In [23]:
type Input = [Int]
type Output = IM.IntMap Int

data ParseResult' a =  Halt' Output | Result' Input a | Unknown' deriving (Show)

instance Functor ParseResult' where
    _ `fmap` Halt' a = Halt' a
    _ `fmap` Unknown' = Unknown'
    f `fmap` Result' i a = Result' i (f a)

isHalt' :: ParseResult' a -> Bool
isHalt' (Halt' _) = True
isHalt' _ = False

isError :: ParseResult' a -> Bool
isError Unknown' = True
isError _ = False

onResult :: ParseResult' a -> 
    (Input -> a -> ParseResult' b) -> 
    ParseResult' b
onResult Unknown' _ = Unknown'
onResult (Halt' i) _ = Halt' i
onResult (Result' i a) k = k i a

onHalt :: ParseResult' a -> Maybe Output
onHalt (Halt' i) = Just i
onHalt _ = Nothing

newtype Parser' a = Parser' (Input -> ParseResult' a) 

parse' :: Parser' a -> Input -> ParseResult' a
parse' (Parser' p) = p

instance Functor Parser' where
    f `fmap` (Parser' p) = Parser' (fmap f . p )

instance Applicative Parser' where
    pure a = Parser' (`Result'` a) -- valueParser
    
    (Parser' f) <*> (Parser' p) = Parser' (\i -> case p i of
        Halt' a -> Halt' a
        Unknown' -> Unknown'
        Result' j a -> (\g -> g a) <$> f j)
        
instance Monad Parser' where
    (Parser' p) >>= f = Parser' (\i ->
        onResult (p i) (\j a -> parse' (f a) j))
    


addParser :: Parser' Output
addParser = Parser' (
    \case
        l@(o : a : b : c : xs) | o == 1 -> 
            let v = IM.fromList $ itoList l
            in
            Result' xs $ 
                maybe 
                    v
                    ((`IM.union` v) . IM.singleton c) 
                    (liftA2 (liftA2 (+)) (!! a) (!! b) l)
        _ -> Unknown')
    
    
prodParser :: Parser' Output
prodParser = Parser' (
    \case 
        l@(o : a : b : c : xs) | o == 2 -> 
            let v = IM.fromList $ itoList l
            in
            Result' xs $ 
                maybe 
                    v 
                    ((`IM.union` v) . IM.singleton c) 
                    (liftA2 (liftA2 (*)) (!! a) (!! b) l) 
        _ -> Unknown')
    
haltParser :: Parser' Output
haltParser = Parser' (
    \case
        l@(o : _) | o == 99 -> Halt' (IM.fromList $ itoList l)
        _ -> Unknown')
        

(|||) :: Parser' a -> Parser' a -> Parser' a
(Parser' p1) ||| (Parser' p2) = Parser' (\i -> let v = p1 i in bool v (p2 i) (isError v))
    

In [ ]:
parse' (addParser ||| prodParser ||| haltParser) [2,2,3,4,5,99]

Result' [5,99] (fromList [(0,2),(1,2),(2,3),(3,4),(4,12),(5,99)])